In [31]:
from pathlib import Path
import shutil
import re

data_path = Path("/home/aleksandernagaj/Milestone/data/LS-VID")
subset = "test"
if not (data_path / f"exported_{subset}").exists():
    (data_path / f"exported_{subset}").mkdir()

# iterate over directories in data path
for person_id in (data_path / f"tracklet_{subset}").iterdir():
    # iterate over files in person directory
    for img in person_id.iterdir():
        im_split = img.name.split("_")
        camera_id = int(im_split[0])
        sequence_n = int(im_split[1])
        if sequence_n % 10 != 0:
            continue

        daytime = re.search(r"[a-z]+", im_split[-1])
        if daytime:
            daytime = daytime.group(0)
            match daytime:
                case "noon":
                    sequence_n += 1000
                case "afternoon":
                    sequence_n += 2000
        
        # save the file in 0001_c1_0000.jpg format in the exported_test directory
        new_name = f"{person_id.name}_c{camera_id}_{sequence_n:0>4d}.jpg"
        # copy file to exported_test/person_id directory
        if not (data_path / f"exported_{subset}" / person_id.name).exists():
            (data_path / f"exported_{subset}" / person_id.name).mkdir()
        shutil.copy(img, data_path / f"exported_{subset}" / person_id.name / new_name)

In [13]:
subset = "val"
for im in (data_path / f"exported_{subset}").glob("**/*.jpg"):
    sequence_n = im.name.split(".")[0].split("_")[-1].lstrip("0")
    if sequence_n == "":
        continue
    if int(sequence_n) % 30 != 0:
        im.unlink()

In [14]:
data_path = Path("/home/aleksandernagaj/Milestone/data/LS-VID/train")
concat_path = Path("/home/aleksandernagaj/Milestone/data/LS-VID/concat")

for im in data_path.glob("**/*.jpg"):
    shutil.copy(im, concat_path / im.name)

In [8]:
people = img_by_person_id(data_path / "query", "*.jpeg")

for person_id, camera_id in people.items():
    # create a directory for each person id with zero padding to 4 digits
    person_dir = data_path / "query" / f"{person_id:04d}"
    person_dir.mkdir(exist_ok=True)
    # copy all images to the new directory, keeping the original files
    for _, images in camera_id.items():
        for image in images:
            shutil.copy(image, person_dir / image.name)
 